# Generate mni subject

1. Imports
2. Download mni template
3. Apply reconall -autorecon1
4. Build surfaces from aseg

In [1]:
from nipype import config
cfg = dict(logging=dict(workflow_level = 'DEBUG'),
           execution={'stop_on_first_crash': False,
                      'hash_method': 'content'})
config.update_config(cfg)

from nilearn.datasets import load_mni152_template,get_data_dirs, fetch_icbm152_2009
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.io import FreeSurferSource

import nibabel as nib
import numpy as np

import os

180424-14:00:49,484 duecredit ERROR:
	 Failed to import duecredit due to No module named 'duecredit'


In [2]:
data_dir = './mni-template/'
template = fetch_icbm152_2009(data_dir=data_dir)
print(template['t1'])


./mni-template/icbm152_2009/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii


In [ ]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

reconall = ReconAll()
reconall.inputs.subject_id = 'mni'
reconall.inputs.directive = 'all'
reconall.inputs.subjects_dir = data_dir
reconall.inputs.T1_files = template['t1']
reconall.inputs.openmp = 2
reconall.inputs.parallel = True
reconall.run()

## Generate surfaces from aseg

In [ ]:
NAMELIST = [
#[1,   'Left-Cerebral-Exterior',                  70,  130, 180, 0],
[2,   'Left-Cerebral-White-Matter',              245, 245, 245, 0],
[3,   'Left-Cerebral-Cortex',                    205, 62,  78,  0],
[4,   'Left-Lateral-Ventricle',                  120, 18,  134, 0],
[5,   'Left-Inf-Lat-Vent',                       196, 58,  250, 0],
#[6,   'Left-Cerebellum-Exterior',                0,   148, 0,   0],
[7,   'Left-Cerebellum-White-Matter',            220, 248, 164, 0],
[8,   'Left-Cerebellum-Cortex',                 230, 148, 34,  0],
[10,  'Left-Thalamus-Proper',                   0,   118, 14,  0],
[11,  'Left-Caudate',                            122, 186, 220, 0],
[12,  'Left-Putamen',                            236, 13,  176, 0],
[13,  'Left-Pallidum',                           12,  48,  255, 0],
[14,  '3rd-Ventricle',                           204, 182, 142, 0],
[15,  '4th-Ventricle',                           42,  204, 164, 0],
[16,  'Brain-Stem',                              119, 159, 176, 0],
[17,  'Left-Hippocampus',                        220, 216, 20,  0],
[18,  'Left-Amygdala',                           103, 255, 255, 0],
#[19,  'Left-Insula',                             80,  196, 98,  0],
[26,  'Left-Accumbens-area',                     255, 165, 0,   0],
[28,  'Left-VentralDC',                          165, 42,  42,  0],
#[40,  'Right-Cerebral-Exterior',                 70,  130, 180, 0],
[41,  'Right-Cerebral-White-Matter',             245, 245, 245, 0],
[42,  'Right-Cerebral-Cortex',                   205, 62,  78, 0],
[43,  'Right-Lateral-Ventricle',                 120, 18,  134, 0],
[44,  'Right-Inf-Lat-Vent',                      196, 58,  250, 0],
#[45,  'Right-Cerebellum-Exterior',               0,   148, 0,   0],
[46,  'Right-Cerebellum-White-Matter',           220, 248, 164, 0],
[47,  'Right-Cerebellum-Cortex',                 230, 148, 34,  0],
[49, 'Right-Thalamus-Proper',                  0,   118, 14,  0],
[50,  'Right-Caudate',                           122, 186, 220, 0],
[51,  'Right-Putamen',                           236, 13,  176, 0],
[52,  'Right-Pallidum',                          13,  48,  255, 0],
[53,  'Right-Hippocampus',                       220, 216, 20,  0],
[54,  'Right-Amygdala',                          103, 255, 255, 0],
#[55,  'Right-Insula',                            80,  196, 98,  0],
[58,  'Right-Accumbens-area',                    255, 165, 0,   0],
[60,  'Right-VentralDC',                         165, 42,  42,  0],
[251, 'CC_Posterior',                            0,   0,   64,  0],
[252, 'CC_Mid_Posterior',                        0,   0,   112, 0],
[253, 'CC_Central',                              0,   0,   160, 0],
[254, 'CC_Mid_Anterior',                         0,   0,   208, 0],
[255, 'CC_Anterior',                             0,   0,   255, 0]]

In [95]:
lut_filepath = 'FreeSurferColorLUT.txt'
lut_array = np.genfromtxt(lut_filepath,
                          dtype=None,
                          usecols=(0, 1, 2, 3, 4, 5),
                          names=['id', 'name', 'R', 'G', 'B', 'A'],
                          encoding='utf-8')
bad_values = [0, 1, 6, 9, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 48, 55, 56, 57, 59, 61, 64, 65, 66, 67, \
              68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 96, 97, 98, 192,  1000]


# print(bad_values)

# Read aseg to extract the labels
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
res = fssource2.run()
q = nib.load(res.outputs.aparc_aseg[0])
volume_values = np.unique(q.get_data())
# print(volume_values)
# Delete all labels that are not in the volume
all_lut_idx = range(0,len(lut_array))

good_values_idx = []
for idx, elem in enumerate(lut_array):
    if elem[0] not in bad_values and elem[0] in volume_values:
        good_values_idx.append(idx)

bad_values_idx = np.setdiff1d(all_lut_idx, good_values_idx)
lut_array = np.delete(lut_array, bad_values_idx)
print(lut_array)

[(   2, 'Left-Cerebral-White-Matter', 245, 245, 245, 0)
 (   4, 'Left-Lateral-Ventricle', 120,  18, 134, 0)
 (   5, 'Left-Inf-Lat-Vent', 196,  58, 250, 0)
 (   7, 'Left-Cerebellum-White-Matter', 220, 248, 164, 0)
 (   8, 'Left-Cerebellum-Cortex', 230, 148,  34, 0)
 (  10, 'Left-Thalamus-Proper',   0, 118,  14, 0)
 (  11, 'Left-Caudate', 122, 186, 220, 0)
 (  12, 'Left-Putamen', 236,  13, 176, 0)
 (  13, 'Left-Pallidum',  12,  48, 255, 0)
 (  14, '3rd-Ventricle', 204, 182, 142, 0)
 (  15, '4th-Ventricle',  42, 204, 164, 0)
 (  16, 'Brain-Stem', 119, 159, 176, 0)
 (  17, 'Left-Hippocampus', 220, 216,  20, 0)
 (  18, 'Left-Amygdala', 103, 255, 255, 0)
 (  26, 'Left-Accumbens-area', 255, 165,   0, 0)
 (  28, 'Left-VentralDC', 165,  42,  42, 0)
 (  41, 'Right-Cerebral-White-Matter',   0, 225,   0, 0)
 (  43, 'Right-Lateral-Ventricle', 120,  18, 134, 0)
 (  44, 'Right-Inf-Lat-Vent', 196,  58, 250, 0)
 (  46, 'Right-Cerebellum-White-Matter', 220, 248, 164, 0)
 (  47, 'Right-Cerebellum-Cortex'

In [96]:
wf = Workflow(name="generate_surfaces_aseg")

fssource = Node(interface=FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni'), name='fssource')
pretess = Node(interface=fs.MRIPretess(subjects_dir=os.path.abspath(data_dir)), name='pretess')
tess =  Node(interface=fs.MRITessellate(subjects_dir=os.path.abspath(data_dir)), name='tess')
smooth =  Node(interface=fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir)), name='smooth')
convert = Node(interface=fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),), name='convert')

wf.connect(fssource, 'aseg', pretess, 'in_filled')
wf.connect(fssource, 'norm', pretess, 'in_norm')
wf.connect(pretess, 'out_file', tess, 'in_file')
wf.connect(tess, 'surface', smooth, 'in_file')
wf.connect(smooth, 'surface', convert, 'in_file')

In [98]:
for l in lut_array:
    if not os.path.exists(os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', l[1]+'.surf.gii')):
        # Pre tessellation
        pretess.inputs.label = l[0]
        pretess.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', l[1]+'_filled.mgz')
        pretess.outputs
        # Tesselation
        tess.inputs.label_value = l[0]
        tess.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp',l[1]+'_nonsmooth' )
        # Smoothing
        smooth.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp',l[1] )
        # Convert
        convert.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', l[1]+'.surf.gii')
        # Run pipeline
        wf.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

180424-15:25:36,354 workflow INFO:
	 Workflow generate_surfaces_aseg settings: ['check', 'execution', 'logging', 'monitoring']
180424-15:25:36,364 workflow INFO:
	 Running serially.
180424-15:25:36,365 workflow INFO:
	 [Node] Setting-up "generate_surfaces_aseg.fssource" in "/var/folders/kf/218x3zrs5yd240h419ybsryh0000gq/T/tmprbgmc0z2/generate_surfaces_aseg/fssource".
180424-15:25:36,371 workflow INFO:
	 [Node] Running "fssource" ("nipype.interfaces.io.FreeSurferSource")
180424-15:25:36,399 workflow INFO:
	 [Node] Finished "generate_surfaces_aseg.fssource".
180424-15:25:36,402 workflow INFO:
	 [Node] Setting-up "generate_surfaces_aseg.pretess" in "/var/folders/kf/218x3zrs5yd240h419ybsryh0000gq/T/tmpyzm_uqds/generate_surfaces_aseg/pretess".
180424-15:25:36,425 workflow INFO:
	 [Node] Running "pretess" ("nipype.interfaces.freesurfer.utils.MRIPretess"), a CommandLine Interface with command:
mri_pretess /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/mri/aseg.mgz 2 /Users

RuntimeError: Workflow did not execute cleanly. Check log for details

## Generte surfaces from other segmentations

In [ ]:
wf = Workflow(name="generate_surfaces_aparc_aseg")

fssource = Node(interface=FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni'), name='fssource')
pretess = Node(interface=fs.MRIPretess(subjects_dir=os.path.abspath(data_dir)), iterfield='in_filled', name='pretess')
tess =  Node(interface=fs.MRITessellate(subjects_dir=os.path.abspath(data_dir)), name='tess')
smooth =  Node(interface=fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir)), name='smooth')
convert = Node(interface=fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),), name='convert')

#wf.connect(fssource, 'aparc_aseg', pretess, 'in_filled')
wf.connect(fssource, 'norm', pretess, 'in_norm')
wf.connect(pretess, 'out_file', tess, 'in_file')
wf.connect(tess, 'surface', smooth, 'in_file')
wf.connect(smooth, 'surface', convert, 'in_file')


In [ ]:
# Load volume to know the different mask values
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
res = fssource2.run()
aparc_aseg_files = res.outputs.aparc_aseg
print(aparc_aseg_files)


for aaf in aparc_aseg_files[1:]:
    
    # Get labels from aseg
    img = nib.load(aaf)
    unique_labels = np.unique(img.get_data())
    
    # Set the aparc_aseg_file as input to pretesselation
    pretess.inputs.in_filled = aaf
    output_folder = aaf.split('/')[-1].replace('.mgz','')
    output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', output_folder)
    
    # Create output if it doesn't exist
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    # Generate surfaces
    for l in unique_labels:
        # Pre tessellation
        pretess.inputs.label = l
        pretess.inputs.out_file = os.path.join(output_folder, str(l)+'_filled.mgz')
        #pretess.outputs

        # Tesselation
        tess.inputs.label_value = l
        tess.inputs.out_file = os.path.join(output_folder,str(l)+'_nonsmooth' )

        # Smoothing
        smooth.inputs.out_file = os.path.join(output_folder,str(l) )

        # Convert
        convert.inputs.out_file = os.path.join(output_folder, str(l)+'.surf.gii')

        # Run pipeline
        wf.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

In [71]:
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
print(fssource2.inputs)
res = fssource2.run()
res.outputs


hemi = both
ignore_exception = False
subject_id = mni
subjects_dir = ./mni-template/




BA_stats = <undefined>
T1 = /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/mri/T1.mgz
annot = ['/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.a2009s.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.DKTatlas.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.BA_exvivo.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.BA_exvivo.thresh.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.a2009s.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.DKTatlas.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.BA_exvivo.annot', '/Users/tgurgu

In [ ]:
pretess.inputs

In [ ]:
!git pull

In [72]:
import nibabel as nib
import numpy as np
q = nib.load(res.outputs.aparc_aseg[0])
np.unique(q.get_data())

array([   0,    2,    4,    5,    7,    8,   10,   11,   12,   13,   14,
         15,   16,   17,   18,   24,   26,   28,   30,   31,   41,   43,
         44,   46,   47,   49,   50,   51,   52,   53,   54,   58,   60,
         62,   63,   77,   85,  251,  252,  253,  254,  255, 1000, 1001,
       1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013,
       1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024,
       1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035,
       2000, 2001, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033,
       2034, 2035], dtype=int32)